In [ ]:
import numpy as np
import pandas as pd
import time
import random

In [ ]:
# read in meta data
problem_instance = "heur040_n_300_m_13358"
metadata = pd.read_csv("data\inst_tuning\\"+problem_instance +".txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

n

In [ ]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

In [ ]:
# create nodes data frame
nodes_from = df.loc[df["e"]==1][["n1","w"]].groupby(['n1']).sum()
nodes_from
nodes_to = df.loc[df["e"]==1][["n2","w"]].groupby(['n2']).sum()
nodes_to

nodes = nodes_from.join(nodes_to, lsuffix='_from', rsuffix='_to', how = 'outer')
nodes['node_impact'] = nodes.w_from.fillna(0) + nodes.w_to.fillna(0)
nodes = nodes.drop(columns=['w_from', 'w_to'])
nodes['current_degree'] = 0
nodes['splex'] = nodes.index
nodes = nodes.reset_index().rename(columns={"index":"node_number"})
nodes

In [ ]:
# create edges data frame
edges = df.copy()
edges['w'] = edges['w'] * (1-(edges['e']*2))
edges['e'] = 0
edges.loc[edges['w'] < 0]

In [ ]:
def is_splex(nodes, plex_number, s) -> bool | pd.DataFrame:
    splex = nodes.loc[nodes["splex"] == plex_number]
    min_degree = len(splex.index) - s
    problem_nodes = splex.loc[splex["current_degree"] < min_degree]
    
    if len(problem_nodes.index) == 0:
        return True
    else:
        return problem_nodes

is_splex(nodes, 2, s)

In [ ]:
def construction_heuristic(nodes, edges, s):
    start = time.time()
    
    existing_edges = edges.loc[edges["w"]<0].sort_values("w")
    remaining_edges = len(existing_edges) # wozu?

    for index, row in existing_edges.iterrows():
        # check if this edge was already added, then we can continue to next one
        if edges.loc[(edges["n1"]==row["n1"]) & (edges["n2"]==row["n2"]), "e"].values[0] == 1:
            continue
        # get plex assignment of both nodes
        n1_plex = nodes.loc[nodes["node_number"]==row["n1"], "splex"].values[0]
        n2_plex = nodes.loc[nodes["node_number"]==row["n2"], "splex"].values[0]
        # all nodes that would be in the plex if we merged it
        nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
        # all edges we would want anyway as they were in original assignment
        other_edges_to_add = edges.loc[(edges["n1"].isin(nodes_in_plex)) & 
                                        (edges["n2"].isin(nodes_in_plex)) & # only edges within the potential splex
                                       (edges["w"]<=0) & # that we want to add anyway
                                       (edges["e"]==0)] #that have not been added yet

        number_of_nodes = len(nodes_in_plex)
        edges_missing = False

        # check if it would be a valid splex if we merge by checking node degrees
        for node in nodes_in_plex:
            node_degree = nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]
            if node_degree < (number_of_nodes - s):
                # if we can reach the node degree by only using the edges we want to add anyway, then everything is fine
                num_potential_edges = other_edges_to_add.loc[(other_edges_to_add["n1"]==node)|
                                                             (other_edges_to_add["n2"]==node)].shape[0]
                if num_potential_edges < number_of_nodes - s - node_degree:
                    edges_missing = True
                    break             

        # if it would be a valid splex, actually merge it
        if not edges_missing:
            # merge them
            nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
            # include all edges we want to add
            for index, oedge in other_edges_to_add.iterrows():
                edges.loc[(edges["n1"]==oedge["n1"]) & (edges["n2"]==oedge["n2"]), "e"] = 1
                # update node info (degree and node impact)
                nodes.loc[nodes["node_number"]==oedge["n1"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n2"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n1"], "node_impact"] -= abs(oedge["w"])
                nodes.loc[nodes["node_number"]==oedge["n2"], "node_impact"] -= abs(oedge["w"])

            remaining_edges -= 1 #wozu?
    print(round(time.time()-start, 2), "seconds")
    return(nodes, edges)

In [ ]:
constr_nodes, constr_edges = construction_heuristic(nodes,edges,s)

In [ ]:
print(len(constr_nodes["splex"].unique()))
constr_nodes

In [ ]:
def randomized_greedy(nodes, edges, s, alpha=0.5, random_seed = None):
    if random_state not None:
        random.seed(random_seed)
    start = time.time()
    
    existing_edges = edges.loc[edges["w"]<0].sort_values("w") # this is our candidate list
    
    while existing_edges.shape[0]>0:
        #build restricted candidate list
        costs_threshold = max(existing_edges["w"]) + alpha * (min(existing_edges["w"]) - max(existing_edges["w"]))
        rcl = existing_edges.loc[existing_edges["w"]>= costs_threshold]
        # pick one edge at random
        row = rcl.sample(1)
            
        # get plex assignment of both nodes
        n1_plex = nodes.loc[nodes["node_number"]==row["n1"].values[0], "splex"].values[0]
        n2_plex = nodes.loc[nodes["node_number"]==row["n2"].values[0], "splex"].values[0]
        # all nodes that would be in the plex if we merged it
        nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
        # all edges we would want anyway as they were in original assignment
        other_edges_to_add = edges.loc[(edges["n1"].isin(nodes_in_plex)) & 
                                        (edges["n2"].isin(nodes_in_plex)) & # only edges within the potential splex
                                       (edges["w"]<=0) & # that we want to add anyway
                                       (edges["e"]==0)] #that have not been added yet

        number_of_nodes = len(nodes_in_plex)
        edges_missing = False

        # check if it would be a valid splex if we merge by checking node degrees
        for node in nodes_in_plex:
            node_degree = nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]
            if node_degree < (number_of_nodes - s):
                # if we can reach the node degree by only using the edges we want to add anyway, then everything is fine
                num_potential_edges = other_edges_to_add.loc[(other_edges_to_add["n1"]==node)|
                                                             (other_edges_to_add["n2"]==node)].shape[0]
                if num_potential_edges < number_of_nodes - s - node_degree:
                    edges_missing = True
                    break             

        # if it would be a valid splex, actually merge it
        if not edges_missing:
            # merge them
            nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
            # include all edges we want to add
            for index, oedge in other_edges_to_add.iterrows():
                edges.loc[(edges["n1"]==oedge["n1"]) & (edges["n2"]==oedge["n2"]), "e"] = 1
                # update node info (degree and node impact)
                nodes.loc[nodes["node_number"]==oedge["n1"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n2"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n1"], "node_impact"] -= abs(oedge["w"])
                nodes.loc[nodes["node_number"]==oedge["n2"], "node_impact"] -= abs(oedge["w"])

        # remove the edge from the candidate list
        existing_edges = existing_edges.drop(existing_edges[(existing_edges.n1 == row.n1.values[0])&
                                                            (existing_edges.n2 == row.n2.values[0])].index)
    print(round(time.time()-start, 2), "seconds")
    return(nodes, edges)

In [ ]:
rand_nodes, rand_edges = randomized_greedy(nodes, edges, s, alpha = 0.75)

In [ ]:
print(len(rand_nodes["splex"].unique()))
rand_nodes

In [ ]:
final_solution = edges.loc[((edges["e"]==0)&(edges["w"]<=0))|
                           ((edges["e"]==1)&(edges["w"]>0)), ["n1", "n2"]]

header = problem_instance + '\n' + final_solution.to_string(header=False, index=False, index_names=False, justify='left', col_space=1)

# Save the combined string to a text file
with open("output/"+problem_instance+".txt", 'w') as file:
    file.write(header)